# Multi-Head Attention

Multi-Head Attention is an important part of all Transformer-based models.
This tutorial will show how to write it and how to then optimize it.

In [1]:
import time
import torch
import ttnn

torch.manual_seed(0)

device_id = 0
device = ttnn.open(device_id)

                  Metal | INFO     | Initializing device 0
                 Device | INFO     | Opening user mode device driver
2023-12-05 03:29:37.684 | INFO     | SiliconDriver   - Detected 1 PCI device
2023-12-05 03:29:37.695 | WARNING  | SiliconDriver   - init_detect_tt_device_numanodes(): Could not determine NumaNodeSet for TT device (physical_device_id: 0 pci_bus_id: 0000:00:08.0)
2023-12-05 03:29:37.695 | WARNING  | SiliconDriver   - Could not find NumaNodeSet for TT Device (physical_device_id: 0 pci_bus_id: 0000:00:08.0)
2023-12-05 03:29:37.695 | INFO     | SiliconDriver   - Using 1 Hugepages/NumHostMemChannels for TTDevice (pci_interface_id: 0 device_id: 0xfaca revision: 0)
2023-12-05 03:29:37.696 | WARNING  | SiliconDriver   - bind_area_memory_nodeset(): Unable to determine TT Device to NumaNode mapping for physical_device_id: 0. Skipping membind.
---- ttSiliconDevice::init_hugepage: bind_area_to_memory_nodeset() failed (physical_device_id: 0 ch: 0). Hugepage allocation is no

## Enable program cache

In [2]:
ttnn.enable_program_cache()

                     Op | INFO     | Program Cache: enabled.


## Write Multi-Head Attention using ttnn

Multi-head can be implemented in `torch` using just 6 operations:
1. `torch.matmul`
2. `torch.add`
3. `torch.reshape`
4. `torch.permute`
5. `torch.mul`
6. `torch.softmax`

`ttnn` provides the exact same APIs to do that and therefore multi-head attention can be implemented as shown below:

In [3]:
def multi_head_attention(
    hidden_states,
    query_weight,
    query_bias,
    key_weight,
    key_bias,
    value_weight,
    value_bias,
    output_weight,
    output_bias,
    *,
    head_size,
):
    batch_size, sequence_size, hidden_size = hidden_states.shape
    num_heads = hidden_size // head_size

    query = hidden_states @ query_weight
    query = query + query_bias
    query = ttnn.reshape(query, (batch_size, sequence_size, num_heads, head_size))
    query = ttnn.permute(query, (0, 2, 1, 3))

    key = hidden_states @ key_weight
    key = key + key_bias
    key = ttnn.reshape(key, (batch_size, sequence_size, num_heads, head_size))
    key = ttnn.permute(key, (0, 2, 3, 1))

    value = hidden_states @ value_weight
    value = value + value_bias
    value = ttnn.reshape(value, (batch_size, sequence_size, num_heads, head_size))
    value = ttnn.permute(value, (0, 2, 1, 3))

    attention_scores = query @ key
    attention_scores = attention_scores * (1 / (head_size**0.5))
    attention_probs = ttnn.softmax(attention_scores, dim=-1)

    context_layer = attention_probs @ value
    context_layer = ttnn.permute(context_layer, (0, 2, 1, 3))
    context_layer = ttnn.reshape(context_layer, (batch_size, sequence_size, hidden_size))

    self_output = context_layer @ output_weight
    self_output = self_output + output_bias

    return self_output

Now that the model is written, let's create input tensors to run it and test it

## Configuration

In [4]:
batch_size = 8
sequence_size = 384
num_heads = 16
head_size = 64
hidden_size = num_heads * head_size

## Initialize activations and weights using torch

In [5]:
torch_hidden_states = torch.randn((batch_size, sequence_size, hidden_size), dtype=torch.bfloat16)
torch_query_weight = torch.randn((hidden_size, hidden_size), dtype=torch.bfloat16)
torch_query_bias = torch.randn((hidden_size,), dtype=torch.bfloat16)
torch_key_weight = torch.randn((hidden_size, hidden_size), dtype=torch.bfloat16)
torch_key_bias = torch.randn((hidden_size,), dtype=torch.bfloat16)
torch_value_weight = torch.randn((hidden_size, hidden_size), dtype=torch.bfloat16)
torch_value_bias = torch.randn((hidden_size,), dtype=torch.bfloat16)
torch_output_weight = torch.randn((hidden_size, hidden_size), dtype=torch.bfloat16)
torch_output_bias = torch.randn((hidden_size,), dtype=torch.bfloat16)

## Convert activations and weights to ttnn

In [6]:
hidden_states = ttnn.from_torch(torch_hidden_states)
query_weight = ttnn.from_torch(torch_query_weight)
query_bias = ttnn.from_torch(torch_query_bias)
key_weight = ttnn.from_torch(torch_key_weight)
key_bias = ttnn.from_torch(torch_key_bias)
value_weight = ttnn.from_torch(torch_value_weight)
value_bias = ttnn.from_torch(torch_value_bias)
output_weight = ttnn.from_torch(torch_output_weight)
output_bias = ttnn.from_torch(torch_output_bias)

## Move activations and weights to device

In [7]:
hidden_states = ttnn.to_device(hidden_states, device)
query_weight = ttnn.to_device(query_weight, device)
query_bias = ttnn.to_device(query_bias, device, memory_config=ttnn.L1_MEMORY_CONFIG)
key_weight = ttnn.to_device(key_weight, device)
key_bias = ttnn.to_device(key_bias, device, memory_config=ttnn.L1_MEMORY_CONFIG)
value_weight = ttnn.to_device(value_weight, device)
value_bias = ttnn.to_device(value_bias, device, memory_config=ttnn.L1_MEMORY_CONFIG)
output_weight = ttnn.to_device(output_weight, device)
output_bias = ttnn.to_device(output_bias, device, memory_config=ttnn.L1_MEMORY_CONFIG)

## Run the first iteration of Multi-Head Attention

In [8]:
start = time.time()
multi_head_attention(
    hidden_states,
    query_weight,
    query_bias,
    key_weight,
    key_bias,
    value_weight,
    value_bias,
    output_weight,
    output_bias,
    head_size=head_size,
)
end = time.time()
duration = end - start

                     Op | INFO     | Program of Operation tt::tt_metal::Tilize                               finished in     0.509617611 seconds
                     Op | INFO     | Program of Operation tt::tt_metal::Tilize                               finished in     0.311836105 seconds
                     Op | INFO     | Program of Operation tt::tt_metal::Matmul                               finished in     0.559607143 seconds
                     Op | INFO     | Program of Operation tt::tt_metal::TilizeWithValPadding                 finished in      0.54272325 seconds
                     Op | INFO     | Program of Operation tt::tt_metal::EltwiseBinaryBroadcast               finished in      0.47534169 seconds
                     Op | INFO     | Program of Operation tt::tt_metal::Untilize                             finished in     0.452674071 seconds
                     Op | INFO     | Program of Operation tt::tt_metal::TilizeWithValPadding                 finished in     0.597

In [9]:
print(f"Multi-head attention ran in {duration} seconds for the first iteration")

Multi-head attention ran in 8.083523273468018 seconds for the first iteration


## Run a subsequent iteration of Multi-Head Attention

In [10]:
start = time.time()
output = multi_head_attention(
    hidden_states,
    query_weight,
    query_bias,
    key_weight,
    key_bias,
    value_weight,
    value_bias,
    output_weight,
    output_bias,
    head_size=head_size,
)
end = time.time()
duration = end - start

                     Op | INFO     | Program of Operation tt::tt_metal::Tilize                               finished in     0.004702303 seconds
                     Op | INFO     | Program of Operation tt::tt_metal::Tilize                               finished in     0.001619931 seconds
                     Op | INFO     | Program of Operation tt::tt_metal::Matmul                               finished in     0.001225913 seconds
                     Op | INFO     | Program of Operation tt::tt_metal::TilizeWithValPadding                 finished in     0.000985024 seconds
                     Op | INFO     | Program of Operation tt::tt_metal::EltwiseBinaryBroadcast               finished in     0.000815475 seconds
                     Op | INFO     | Program of Operation tt::tt_metal::Untilize                             finished in     0.006004395 seconds
                     Op | INFO     | Program of Operation tt::tt_metal::TilizeWithValPadding                 finished in     0.159

In [11]:
print(f"Multi-head attention ran in {duration} seconds for the subsequent iteration because of the program cache")

Multi-head attention ran in 0.6267373561859131 seconds for the subsequent iteration because of the program cache


## Write optimized version of Multi-Head Attention

Optimized version of the multi-head attention can be written by:
- Tilizing all of the tensors ahead of time
- Using more performant matmuls that fuse bias and specify the number of cores they execute on
- Putting every tensor into L1
- Using bfloat8_b data_type
- Using custom `nlp` operations instead of `ttnn.permute` and `ttnn.reshape`

`ttnn.free` calls are needed because otherwise, the cores on the device will run out of the L1 memory

In [12]:
def optimized_multi_head_attention(
    hidden_states,
    fused_qkv_weight,
    fused_qkv_bias,
    self_output_weight,
    self_output_bias,
    *,
    head_size,
    num_cores_x=12,
):
    batch_size, *_ = hidden_states.shape
    
    hidden_states = ttnn.to_layout(hidden_states, ttnn.TILE_LAYOUT)

    fused_qkv_output = ttnn.linear(
        hidden_states,
        fused_qkv_weight,
        bias=fused_qkv_bias,
        memory_config=ttnn.L1_MEMORY_CONFIG,
        dtype=ttnn.bfloat8_b,
        core_grid=(batch_size, num_cores_x),
    )

    (
        query,
        key,
        value,
    ) = ttnn.nlp.split_fused_qkv_and_split_heads(
        fused_qkv_output,
        memory_config=ttnn.L1_MEMORY_CONFIG,
        core_grid=(batch_size, num_cores_x),
    )
    ttnn.free(fused_qkv_output)

    attention_scores = ttnn.matmul(
        query,
        key,
        memory_config=ttnn.L1_MEMORY_CONFIG,
        dtype=ttnn.bfloat16,
        core_grid=(batch_size, num_cores_x),
    )
    ttnn.free(query)
    ttnn.free(key)

    attention_probs = ttnn.nlp.attention_softmax(attention_scores, attention_mask=None, head_size=head_size)

    context_layer = ttnn.matmul(
        attention_probs,
        value,
        memory_config=ttnn.L1_MEMORY_CONFIG,
        dtype=ttnn.bfloat8_b,
        core_grid=(batch_size, num_cores_x),
    )
    ttnn.free(attention_probs)

    context_layer = ttnn.nlp.concatenate_heads(
        context_layer,
        memory_config=ttnn.L1_MEMORY_CONFIG,
        core_grid=(batch_size, num_cores_x),
    )

    self_output = ttnn.linear(
        context_layer,
        self_output_weight,
        bias=self_output_bias,
        memory_config=ttnn.L1_MEMORY_CONFIG,
        dtype=ttnn.bfloat16,
        core_grid=(batch_size, num_cores_x),
    )
    ttnn.free(context_layer)

    return self_output

## Pre-process the parameters of the optimized model

1. Fuse QKV weights and biases
2. Reshape and tilize for the optimized operations using preprocess_linear_weight and preprocess_linear_bias
3. Move to device

In [13]:
from ttnn.model_preprocessing import (
    ParametersConfig,
    preprocess_linear_bias,
    preprocess_linear_weight,
)

torch_qkv_weight = torch.cat([torch_query_weight, torch_key_weight, torch_value_weight], dim=-1)
torch_qkv_bias = torch.cat([torch_query_bias, torch_key_bias, torch_value_bias], dim=-1)

parameters_config = ParametersConfig()
qkv_weight = preprocess_linear_weight(parameters_config, torch_qkv_weight.T)
qkv_bias = preprocess_linear_bias(parameters_config, torch_qkv_bias)
output_weight = preprocess_linear_weight(parameters_config, torch_output_weight.T)
output_bias = preprocess_linear_bias(parameters_config, torch_output_bias)

qkv_weight = ttnn.to_device(qkv_weight, device)
qkv_bias = ttnn.to_device(qkv_bias, device, memory_config=ttnn.L1_MEMORY_CONFIG)
output_weight = ttnn.to_device(output_weight, device)
output_bias = ttnn.to_device(output_bias, device, memory_config=ttnn.L1_MEMORY_CONFIG)

## Run the first iteration of the optimized Multi-Head Attention

In [14]:
start = time.time()
hidden_states = ttnn.to_layout(hidden_states, ttnn.TILE_LAYOUT)
optimized_output = optimized_multi_head_attention(
    hidden_states,
    qkv_weight,
    qkv_bias,
    output_weight,
    output_bias,
    head_size=head_size,
)
end = time.time()
duration = end - start

                     Op | INFO     | Program of Operation tt::tt_metal::Tilize                               finished in      0.00520659 seconds
                     Op | INFO     | Program of Operation tt::operations::primary::Matmul                    finished in     0.722743889 seconds
                     Op | INFO     | Program of Operation tt::operations::primary::transformers::SplitFusedQKVAndSplitHeads finished in     0.461215513 seconds
                     Op | INFO     | Program of Operation tt::operations::primary::Matmul                    finished in     0.604019846 seconds
                     Op | INFO     | Program of Operation tt::tt_metal::EltwiseBinaryBroadcast               finished in     0.391954323 seconds
                     Op | INFO     | Program of Operation tt::operations::primary::Softmax                   finished in     0.001912639 seconds
                     Op | INFO     | Program of Operation tt::operations::primary::Matmul                    finish

In [15]:
print(f"Optimized multi-head attention ran in {duration} seconds for the first iteration")

Optimized multi-head attention ran in 4.0068464279174805 seconds for the first iteration


## Run a subsequent iteration of the optimized Multi-Head Attention

In [16]:
start = time.time()
optimized_output = optimized_multi_head_attention(
    hidden_states,
    qkv_weight,
    qkv_bias,
    output_weight,
    output_bias,
    head_size=head_size,
)
end = time.time()
duration = end - start

                     Op | INFO     | Program of Operation tt::operations::primary::Matmul                    finished in     0.000584427 seconds
                     Op | INFO     | Program of Operation tt::operations::primary::transformers::SplitFusedQKVAndSplitHeads finished in     0.000196729 seconds
                     Op | INFO     | Program of Operation tt::operations::primary::Matmul                    finished in     0.000440207 seconds
                     Op | INFO     | Program of Operation tt::tt_metal::EltwiseBinaryBroadcast               finished in     0.001433742 seconds
                     Op | INFO     | Program of Operation tt::operations::primary::Softmax                   finished in     0.001892659 seconds
                     Op | INFO     | Program of Operation tt::operations::primary::Matmul                    finished in     0.000821455 seconds
                     Op | INFO     | Program of Operation tt::operations::primary::transformers::ConcatenateHeads f

In [17]:
print(f"Optimized multi-head attention ran in {duration} seconds for the subsequent iteration because of the program cache")

Optimized multi-head attention ran in 0.007799386978149414 seconds for the subsequent iteration because of the program cache


Note that the optimized multi-head attention is 2 orders of magnitude faster than the initial version

## Check that the output of the optimized version matches the output of the original implementation

In [18]:
output = ttnn.to_layout(output, ttnn.ROW_MAJOR_LAYOUT)
output = ttnn.from_device(output)
torch_output = ttnn.to_torch(output)

optimized_output = ttnn.to_layout(optimized_output, ttnn.ROW_MAJOR_LAYOUT)
optimized_output = ttnn.from_device(optimized_output)
torch_optimized_output = ttnn.to_torch(optimized_output)

assert torch.allclose(torch_output, torch_optimized_output)

                     Op | INFO     | Program of Operation tt::tt_metal::Untilize                             finished in     0.006005305 seconds
                     Op | INFO     | Program of Operation tt::tt_metal::Untilize                             finished in     0.373713348 seconds


## Close the device

In [19]:
ttnn.close(device)

                  Metal | INFO     | Closing device 0
